In [74]:
import glob
import os

projects = os.listdir("data/projects")
last_commits = glob.glob("data/projects_last_commit/*.json")

print(f"Analyzed projects: {len(projects)}")
print(f"Last commit extracted for projects: {len(last_commits)}")

Analyzed projects: 1079
Last commit extracted for projects: 1109


In [75]:
for project in projects:
    last_commit_file = f"data/projects_last_commit/{project}_last_commit.json"
    if last_commit_file not in last_commits:
        print(f"Missing last commit for project: {project}")
        shutil.rmtree(last_commit_file)

In [76]:
import shutil

for project in projects:
    json_files = glob.glob(f"data/projects/{project}/*.json")
    if any(f for f in json_files if f.endswith("last_commit.json")):
        print(project)

    #if len(json_files) == 1 and json_files[0].endswith("last_commit.json"):
    #    shutil.rmtree(f"data/projects/{project}")

lucide-icons_lucide


In [ ]:
for project in projects:
    last_commit_file = f"data/projects_last_commit/{project}_last_commit.json"
    if last_commit_file not in last_commits:
        print(f"Missing last commit for project: {project}")

In [30]:
import pandas as pd

df = pd.read_csv("data/projects_final.csv")

full_names = []

for index, row in df.iterrows():
    name = row["name"]
    full_name = row["full_name"].replace("/", "_")
    full_names.append((name, full_name))

for name, full_name in full_names:
    print(name, full_name)

react facebook_react
linux torvalds_linux
tensorflow tensorflow_tensorflow
ohmyzsh ohmyzsh_ohmyzsh
AutoGPT Significant-Gravitas_AutoGPT
vscode microsoft_vscode
bootstrap twbs_bootstrap
flutter flutter_flutter
transformers huggingface_transformers
ollama ollama_ollama
youtube-dl ytdl-org_youtube-dl
next.js vercel_next.js
go golang_go
scrcpy Genymobile_scrcpy
react-native facebook_react-native
PowerToys microsoft_PowerToys
electron electron_electron
kubernetes kubernetes_kubernetes
yt-dlp yt-dlp_yt-dlp
hello-algo krahets_hello-algo
node nodejs_node
langchain langchain-ai_langchain
n8n n8n-io_n8n
axios axios_axios
three.js mrdoob_three.js
TypeScript microsoft_TypeScript
rust rust-lang_rust
dify langgenius_dify
deno denoland_deno
excalidraw excalidraw_excalidraw
open-webui open-webui_open-webui
terminal microsoft_terminal
angular angular_angular
godot godotengine_godot
DeepSeek-V3 deepseek-ai_DeepSeek-V3
material-ui mui_material-ui
frp fatedier_frp
ant-design ant-design_ant-design
tauri ta

In [4]:
missing_projects = []

for name, full_name in full_names:
    is_available = False

    for project in last_commits:
        file_name = project.split("/")[-1]
        project_name = file_name.split("_last_commit.json")[0]

        if project_name == name or project_name == full_name:
            is_available = True
            break    

    if not is_available:
        missing_projects.append((name, full_name))

print("Missing projects: ", len(missing_projects))
for project in missing_projects:
    print(project)

Missing projects:  32
('AutoGPT', 'Significant-Gravitas_AutoGPT')
('supabase', 'supabase_supabase')
('grafana', 'grafana_grafana')
('ionic-framework', 'ionic-team_ionic-framework')
('appwrite', 'appwrite_appwrite')
('DefinitelyTyped', 'DefinitelyTyped_DefinitelyTyped')
('bulma', 'jgthms_bulma')
('expo', 'expo_expo')
('photoprism', 'photoprism_photoprism')
('ray', 'ray-project_ray')
('lapce', 'lapce_lapce')
('lottie-android', 'airbnb_lottie-android')
('chatbox', 'chatboxai_chatbox')
('mattermost', 'mattermost_mattermost')
('symfony', 'symfony_symfony')
('GPT_API_free', 'chatanywhere_GPT_API_free')
('server', 'nextcloud_server')
('medusa', 'medusajs_medusa')
('Telegram', 'DrKLO_Telegram')
('erpnext', 'frappe_erpnext')
('vcpkg', 'microsoft_vcpkg')
('qdrant', 'qdrant_qdrant')
('authelia', 'authelia_authelia')
('gitlabhq', 'gitlabhq_gitlabhq')
('difftastic', 'Wilfred_difftastic')
('jaeger', 'jaegertracing_jaeger')
('roslyn', 'dotnet_roslyn')
('fluentui', 'microsoft_fluentui')
('qmk_firmware

In [36]:
dublicated_projects = []

for name, full_name in full_names:
    count = 0
    for project in last_commits:
        file_name = project.split("/")[-1]
        project_name = file_name.split("_last_commit.json")[0]

        if project_name == name or project_name == full_name:
            print(project_name, name, full_name)
            count += 1    

    if count > 1:
        dublicated_projects.append((name, full_name))
        break


print("Dublicated projects: ", len(dublicated_projects))
for project in dublicated_projects:
    print(project)

react react facebook_react
torvalds_linux linux torvalds_linux
tensorflow tensorflow tensorflow_tensorflow
ohmyzsh ohmyzsh ohmyzsh_ohmyzsh
microsoft_vscode vscode microsoft_vscode
bootstrap bootstrap twbs_bootstrap
flutter_flutter flutter flutter_flutter
flutter flutter flutter_flutter
Dublicated projects:  1
('flutter', 'flutter_flutter')


In [5]:
for missing_project in missing_projects:
    name_parts = missing_project[1].split("_")
    print(f"python3 /tmp/ssimon/config-space/experiments/analysis.py --url=https://github.com/{name_parts[0]}/{name_parts[1]} --name={name_parts[1]} --last_commit=True")



python3 /tmp/ssimon/config-space/experiments/analysis.py --url=https://github.com/Significant-Gravitas/AutoGPT --name=AutoGPT --last_commit=True
python3 /tmp/ssimon/config-space/experiments/analysis.py --url=https://github.com/supabase/supabase --name=supabase --last_commit=True
python3 /tmp/ssimon/config-space/experiments/analysis.py --url=https://github.com/grafana/grafana --name=grafana --last_commit=True
python3 /tmp/ssimon/config-space/experiments/analysis.py --url=https://github.com/ionic-team/ionic-framework --name=ionic-framework --last_commit=True
python3 /tmp/ssimon/config-space/experiments/analysis.py --url=https://github.com/appwrite/appwrite --name=appwrite --last_commit=True
python3 /tmp/ssimon/config-space/experiments/analysis.py --url=https://github.com/DefinitelyTyped/DefinitelyTyped --name=DefinitelyTyped --last_commit=True
python3 /tmp/ssimon/config-space/experiments/analysis.py --url=https://github.com/jgthms/bulma --name=bulma --last_commit=True
python3 /tmp/ssimon

In [1]:
import glob

files = glob.glob("data/projects_contributors/*.csv")

print(f"Total contributor files: {len(files)}")

Total contributor files: 1022


In [2]:
import pandas as pd

df = pd.read_csv("data/projects_final.csv")

#for index, row in df.iterrows():
#    print(row["full_name"], row["language"])


languages = df["language"].unique()
print("Languages:", languages)

Languages: ['JavaScript' 'C' 'C++' 'Shell' 'Python' 'TypeScript' 'MDX' 'Dart' 'Go'
 'C#' 'Java' 'Jupyter Notebook' 'Rust' 'Vim Script' 'Blade' 'Clojure'
 'Zig' 'Assembly' 'Ruby' 'Kotlin' 'CSS' 'Julia' 'Astro' 'HTML' 'PHP'
 'Roff' 'Swift' 'Scala' 'Vue' 'Lua' 'Haskell' 'Svelte' 'V' 'PowerShell'
 'Markdown' 'LLVM' 'Elixir' 'Cuda' 'Solidity' 'Objective-C' 'CMake' 'SCSS'
 'Mojo' 'Emacs Lisp' 'OCaml' 'Perl' 'Nix' 'Starlark' nan 'Crystal'
 'Dockerfile' 'Meson' 'Groovy']


In [4]:
import pandas as pd

data_file = "data/technological/aggregated_technology_utilization.csv"

df = pd.read_csv(data_file) 

print(df.columns)

df = df.sort_values(by=["num_projects"], ascending=False)

df.head(50)

Index(['technology', 'ground_truth', 'num_projects', 'avg_files_per_project',
       'avg_options_set_total', 'avg_options_set_unique'],
      dtype='object')


,technology,ground_truth,num_projects,avg_files_per_project,avg_options_set_total,avg_options_set_unique
50,github-action,NaN,985,8.64,410.04,181.10
49,github issues,NaN,644,2.85,76.16,12.30
88,nodejs,42.0,603,19.27,4504.58,3536.99
35,docker,18.0,468,3.93,53.78,6.89
48,github funding,NaN,412,1.01,1.91,1.66
32,dependabot,NaN,384,1.03,15.80,6.63
122,tsconfig,129.0,335,13.84,169.18,37.50
46,github,NaN,287,1.57,41.30,14.19
92,poetry,42.0,213,3.23,80.10,47.87
129,vscode,NaN,199,2.60,30.75,22.12


In [5]:
import pandas as pd 

data_file = "data/technological/utilization/1Panel_technology_utilization.csv"

df = pd.read_csv(data_file)

df.head()

,Technology,Total Options,Number of Files,Options Set (Total),Options Set (Unique),Matched Options,Percentage Used
0,github issues,NaN,3,59,14,NaN,NaN
1,github-action,NaN,8,128,69,NaN,NaN
2,goreleaser,NaN,1,51,21,NaN,NaN
3,vscode,NaN,1,1,1,NaN,NaN
4,swagger,NaN,1,8728,8667,NaN,NaN
